### Pinchman hooks ### 

In [1]:
import cv2
import pyautogui
from time import time
from math import hypot
import mediapipe as mp
import matplotlib.pyplot as plt
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np
from time import sleep


In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

hand_image = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=1)
hand_video = mp_hands.Hands(static_image_mode=False, model_complexity=1, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7, max_num_hands=1)

In [3]:
def detectHands(image, hand_fn, draw=False, display=False):
    # Create a copy of the input image.
    output_image = image.copy()
    
    # Convert the image from BGR into RGB format.
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Perform the Pose Detection.
    results = hand_fn.process(imageRGB)
    
    if results.multi_hand_landmarks and draw:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(output_image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(250, 44, 250), thickness=2, circle_radius=2),
                                         )
    
    if display:
        plt.figure(figsize=[22,22])
        plt.subplot(121);plt.imshow(image[:,:,::-1]);plt.title("Original Image");plt.axis('off')
        plt.subplot(122);plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off')
    else:
        return output_image, results

In [4]:
def checkPinch(image, results, draw=False, display=False):
    '''
        checks if the first finger and thumb are pinching....for starting the game
    '''
    height, width, _ = image.shape
    output_image = image.copy()
    
    one_hand = results.multi_hand_landmarks[0].landmark
    
    # print(one_hand.landmark[4])
    # return 
    
    thumbtip_landmark = (one_hand[4].x*width,
                      one_hand[4].y*height)
    
    indextip_landmark = (one_hand[8].x*width,
                      one_hand[8].y*height)
    
    x1, y1 = int(indextip_landmark[0]), int(indextip_landmark[1])
    x2, y2 = int(thumbtip_landmark[0]), int(thumbtip_landmark[1])
    
    dist = int(hypot(thumbtip_landmark[0]-indextip_landmark[0],
                      thumbtip_landmark[1]-indextip_landmark[1]))
    
    if dist < 20:
        pinch_status = 'Pinched'
        color = (0, 255, 0)
    else:
        pinch_status = 'Not Pinched'
        color = (255, 0, 0)
        
        
    if draw:
        cv2.putText(output_image, pinch_status, (10,30), cv2.FONT_HERSHEY_PLAIN, 2, color, 3)
        cv2.putText(output_image, f'Distance: {dist}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 2, color, 3) 
        cv2.circle(output_image, (x1, y1), 10, (255, 0, 128), cv2.FILLED)
        cv2.circle(output_image, (x2, y2), 10, (255, 0, 128), cv2.FILLED)
        cv2.line(output_image, (x1, y1), (x2, y2), (255, 0, 128), 3)
        
        
    if display:
        # Display the output image.
        plt.figure(figsize=[10,10])
        plt.imshow(output_image[:,:,::-1]);plt.title("Output Image");plt.axis('off')
    # Otherwise
    else:
        # Return the output image and the classified hands status indicating whether the hands are joined or not.
        return output_image, pinch_status  

In [5]:
camera_video = cv2.VideoCapture(0)
camera_video.set(3, 1280)
camera_video.set(4, 960)

time1 = 0
game_started = False

x_pos_index = 1 # left-0 , center-1, right-2
y_pos_index = 1 # crouch-0, center-1, jump-2

counter = 0 # for the number of frames we have to pinch to start the game
num_of_frames = 10

MID_Y = None

# pressing space when pinch position is there for n frames
counter = 0
num_of_frames = 5

while camera_video.isOpened():
    ok, frame = camera_video.read()
    
    if not ok: 
        continue
    
    frame = cv2.flip(frame, 1)
    frame_height, frame_width, _ = frame.shape
    frame, results = detectHands(frame, hand_video, draw=True)
    
    
    
    if results.multi_hand_landmarks:
        if checkPinch(frame, results)[1] == 'Pinched':
            cv2.putText(frame, 'Space Pressed', (10, 50), cv2.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 3)
            pyautogui.keyDown('space')
        else:
            cv2.putText(frame, 'Space Not Pressed', (10, 50), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 3)
            pyautogui.keyUp('space')
        
        
    time2 = time()
    if (time2 - time1) > 0:
        frames_per_second = 1.0 / (time2 - time1)
        cv2.putText(frame, 'FPS: {:.1f}'.format(frames_per_second), (10, 30), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 3)
    time1 = time2
    
    # Display the frame.            
    cv2.imshow('Hookman through pinch', frame)
    k = cv2.waitKey(1) & 0xFF     
    if(k == 27):
        break
 
# Release the VideoCapture Object and close the windows.                  
camera_video.release()
cv2.destroyAllWindows()
                        